IMPORTING DEPENDECIES AND LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go

LOADING THE DATA

In [ ]:
matches_df = pd.read_csv('matches.csv')
deliveries_df = pd.read_csv('deliveries.csv')

In [ ]:
matches_df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [ ]:
deliveries_df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [ ]:
deliveries_df.shape

(260920, 17)

In [ ]:
for col in matches_df.columns:
  if col != 'date' and col != 'id':
    print(f"{col} has unique values: ",matches_df[col].unique())

season has unique values:  ['2007/08' '2009' '2009/10' '2011' '2012' '2013' '2014' '2015' '2016'
 '2017' '2018' '2019' '2020/21' '2021' '2022' '2023' '2024']
city has unique values:  ['Bangalore' 'Chandigarh' 'Delhi' 'Mumbai' 'Kolkata' 'Jaipur' 'Hyderabad'
 'Chennai' 'Cape Town' 'Port Elizabeth' 'Durban' 'Centurion' 'East London'
 'Johannesburg' 'Kimberley' 'Bloemfontein' 'Ahmedabad' 'Cuttack' 'Nagpur'
 'Dharamsala' 'Kochi' 'Indore' 'Visakhapatnam' 'Pune' 'Raipur' 'Ranchi'
 'Abu Dhabi' nan 'Rajkot' 'Kanpur' 'Bengaluru' 'Dubai' 'Sharjah'
 'Navi Mumbai' 'Lucknow' 'Guwahati' 'Mohali']
match_type has unique values:  ['League' 'Semi Final' 'Final' '3rd Place Play-Off' 'Qualifier 1'
 'Elimination Final' 'Qualifier 2' 'Eliminator']
player_of_match has unique values:  ['BB McCullum' 'MEK Hussey' 'MF Maharoof' 'MV Boucher' 'DJ Hussey'
 'SR Watson' 'V Sehwag' 'ML Hayden' 'YK Pathan' 'KC Sangakkara' 'JDP Oram'
 'AC Gilchrist' 'SM Katich' 'MS Dhoni' 'ST Jayasuriya' 'GD McGrath'
 'SE Marsh' 'SA Asn

There doesn't seems any outlier in the case of runs and overs as target are <= 288 (highest target given) and overs is in between 6 to 20.

MODIFYING TEAM NAMES TO OVERCOME OUTLIERS

In [ ]:

matches_df = matches_df.replace({'Delhi Daredevils':'Delhi Capitals','Deccan Chargers':'Sunrisers Hyderabad','Royal Challengers Bengaluru':'Royal Challengers Bangalore','Punjab Kings':'Kings XI Punjab','Rising Pune Supergiants':'Rising Pune Supergiant'})
deliveries_df = deliveries_df.replace({'Delhi Daredevils':'Delhi Capitals','Deccan Chargers':'Sunrisers Hyderabad','Royal Challengers Bengaluru':'Royal Challengers Bangalore','Punjab Kings':'Kings XI Punjab','Rising Pune Supergiants':'Rising Pune Supergiant'})


MODIFYING SEASON IN A PARTICULAR YEAR TO AVOID ANY ISSUE LIKE 2007/8 TO 2008

In [ ]:
matches_df['season'] = matches_df['season'].replace({'2007/08':'2008','2009/10':'2010','2020/21':'2020'})

CHECKING ABNORMAL ENTRIES IN WINNER COLUMNS OF MATCHES DATA

In [ ]:
# nan values in winner is due to non result of the match which is justifiable
matches_df[matches_df['winner'].isnull()==True]['result']

,result
241,no result
485,no result
511,no result
744,no result
994,no result


CHECKING ABNOMRAL RESULTS IN RESULT OF MATCHES

In [ ]:
matches_df['result'].value_counts()

,count
result,
wickets,578
runs,498
tie,14
no result,5


In [ ]:
matches_df[matches_df['target_runs'].isnull()==True]

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
241,501265,2011,Delhi,2011-05-21,League,NaN,Feroz Shah Kotla,Delhi Capitals,Pune Warriors,Delhi Capitals,bat,NaN,no result,NaN,NaN,NaN,N,NaN,SS Hazare,RJ Tucker
485,829763,2015,Bangalore,2015-04-29,League,NaN,M Chinnaswamy Stadium,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,NaN,no result,NaN,NaN,NaN,N,NaN,JD Cloete,PG Pathak
994,1359519,2023,Lucknow,2023-05-03,League,NaN,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Lucknow Super Giants,Chennai Super Kings,Chennai Super Kings,field,NaN,no result,NaN,NaN,NaN,N,NaN,AK Chaudhary,NA Patwardhan


In [ ]:
# when tie and no result are ther result_m
matches_df[matches_df['result_margin'].isnull() == True]

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
66,392190,2009,Cape Town,2009-04-23,League,YK Pathan,Newlands,Kolkata Knight Riders,Rajasthan Royals,Kolkata Knight Riders,field,Rajasthan Royals,tie,NaN,151.0,20.0,Y,NaN,MR Benson,M Erasmus
130,419121,2010,Chennai,2010-03-21,League,J Theron,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,field,Kings XI Punjab,tie,NaN,137.0,20.0,Y,NaN,K Hariharan,DJ Harper
241,501265,2011,Delhi,2011-05-21,League,NaN,Feroz Shah Kotla,Delhi Capitals,Pune Warriors,Delhi Capitals,bat,NaN,no result,NaN,NaN,NaN,N,NaN,SS Hazare,RJ Tucker
328,598004,2013,Hyderabad,2013-04-07,League,GH Vihari,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,bat,Sunrisers Hyderabad,tie,NaN,131.0,20.0,Y,NaN,AK Chaudhary,S Ravi
342,598017,2013,Bangalore,2013-04-16,League,V Kohli,M Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,field,Royal Challengers Bangalore,tie,NaN,153.0,20.0,Y,NaN,M Erasmus,VA Kulkarni
416,729315,2014,Abu Dhabi,2014-04-29,League,JP Faulkner,Sheikh Zayed Stadium,Kolkata Knight Riders,Rajasthan Royals,Rajasthan Royals,bat,Rajasthan Royals,tie,NaN,153.0,20.0,Y,NaN,Aleem Dar,AK Chaudhary
475,829741,2015,Ahmedabad,2015-04-21,League,SE Marsh,"Sardar Patel Stadium, Motera",Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,field,Kings XI Punjab,tie,NaN,192.0,20.0,Y,NaN,M Erasmus,S Ravi
485,829763,2015,Bangalore,2015-04-29,League,NaN,M Chinnaswamy Stadium,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,NaN,no result,NaN,NaN,NaN,N,NaN,JD Cloete,PG Pathak
511,829813,2015,Bangalore,2015-05-17,League,NaN,M Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Capitals,Royal Challengers Bangalore,field,NaN,no result,NaN,188.0,20.0,N,NaN,HDPK Dharmasena,K Srinivasan
610,1082625,2017,Rajkot,2017-04-29,League,KH Pandya,Saurashtra Cricket Association Stadium,Gujarat Lions,Mumbai Indians,Gujarat Lions,bat,Mumbai Indians,tie,NaN,154.0,20.0,Y,NaN,AK Chaudhary,CB Gaffaney


Removing Super Overs Data

In [ ]:
deliveries_df = deliveries_df[deliveries_df['inning'] != 4]
deliveries_df = deliveries_df[deliveries_df['inning'] != 3]


In [ ]:
deliveries_df.shape

(260771, 17)

MATCH PLAYED VS WINNING PERCENTAGE

In [ ]:
# Plot Matches Played and Winning Percentages

# find match played by team
total_matches_played1 = matches_df.groupby('team1')['id'].count().reset_index()
total_matches_played1.columns = ['Team', 'Matches Played']
total_matches_played2 = matches_df.groupby('team2')['id'].count().reset_index()
total_matches_played2.columns = ['Team', 'Matches Played']
total_matches_played = pd.concat([total_matches_played1, total_matches_played2],axis=0)
total_matches_played = total_matches_played.groupby('Team')['Matches Played'].sum().reset_index()
total_matches_played = total_matches_played.sort_values(by='Matches Played', ascending=False)

# find winning matches.(null values has taken care by the pandas itself)
total_matches_played['total_matches_won'] = matches_df.groupby('winner')['id'].count().reset_index()['id']
total_matches_played['winning_percentage'] = (total_matches_played['total_matches_won'] / total_matches_played['Matches Played']) * 100

In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=total_matches_played['Team'],
    y=total_matches_played['Matches Played'],
    name='Matches Played',
    mode='lines+text',  # Changed to 'lines+text'
    text=total_matches_played['Matches Played'],
    textposition='top center'
))

fig.add_trace(go.Scatter(
    x=total_matches_played['Team'],
    y=total_matches_played['winning_percentage'],
    name='Winning Percentage',
    mode='lines+text',  # Changed to 'lines+text'
    text=total_matches_played['winning_percentage'].round(2),
    textposition='top center'
))

fig.update_layout(title='Matches Played and Winning Percentage', xaxis_title='Team')
fig.show()

In [ ]:
no_result_matches_id = matches_df[matches_df['result']=='no result']['id']
resulted_matches = deliveries_df[~deliveries_df['match_id'].isin(no_result_matches_id.tolist())]
resulted_matches = resulted_matches.merge(
    matches_df[matches_df['method']=='D/L'][['method','id']],
    left_on = 'match_id',
    right_on = 'id',
    how='left'
)

In [ ]:
match_summary_df = resulted_matches.groupby(['match_id','batting_team']).agg(
  total_runs=('total_runs', 'sum'),
  # number of legal balls
  ball=('batsman_runs',lambda x: x[~resulted_matches.loc[x.index, 'extras_type'].isin(['wides','noballs'])].count()),
  # number of 4s and 6s
  num_4s=('batsman_runs', lambda x: (x == 4).sum()),
  num_6s=('batsman_runs', lambda x: (x == 6).sum()),
  # include only first 6 overs for power play
  powerplay_score=('total_runs', lambda x: x[resulted_matches.loc[x.index,'over'].isin(range(0,6))].sum()),
  # include last 5 overs for death over
  death_over_score=('total_runs', lambda x: x[resulted_matches.loc[x.index,'over'].isin(range(16,20))].sum()),
).reset_index()
match_summary_df['run_rate'] = match_summary_df['total_runs'] / (match_summary_df['ball'] / 6)


In [ ]:
# we shouldn't consider powerplay and death over when D/L has applied as the powerplay and death over can't be calculated from given dataset.
match_summary_df = match_summary_df.merge(
    matches_df[matches_df['method']=='D/L'][['method','id']],
    left_on = 'match_id',
    right_on = 'id',
    how='left'
)
match_summary_df['powerplay_score'] = match_summary_df.apply(
    lambda x: np.nan if x['method'] == "D/L" else x['powerplay_score'],
    axis=1
)
match_summary_df['death_over_score'] = match_summary_df.apply(
    lambda x: np.nan if x['method'] == "D/L" else x['death_over_score'],
    axis=1
)
match_summary_df = match_summary_df.drop(["method","id"],axis=1)
match_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2180 entries, 0 to 2179
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_id          2180 non-null   int64  
 1   batting_team      2180 non-null   object 
 2   total_runs        2180 non-null   int64  
 3   ball              2180 non-null   int64  
 4   num_4s            2180 non-null   int64  
 5   num_6s            2180 non-null   int64  
 6   powerplay_score   2138 non-null   float64
 7   death_over_score  2138 non-null   float64
 8   run_rate          2180 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 153.4+ KB


In [ ]:
match_summary_df.head()

,match_id,batting_team,total_runs,ball,num_4s,num_6s,powerplay_score,death_over_score,run_rate
0,335982,Kolkata Knight Riders,222,120,15,14,61.0,64.0,11.100000
1,335982,Royal Challengers Bangalore,82,91,3,3,26.0,0.0,5.406593
2,335983,Chennai Super Kings,240,120,20,16,53.0,71.0,12.000000
3,335983,Kings XI Punjab,207,120,18,9,63.0,31.0,10.350000
4,335984,Delhi Capitals,132,91,18,1,55.0,0.0,8.703297


BOWLING SUMMARY DATAFRAME FOR FURHTER ANALYSIS

In [ ]:
bowling_summary_df = resulted_matches.groupby(['match_id', 'bowling_team']).agg(
    runs_conceded = ('total_runs', 'sum'),
    legal_deliveries = ('ball','count')
).reset_index()

bowling_summary_df['economy_rate'] = bowling_summary_df['runs_conceded'] / (bowling_summary_df['legal_deliveries'] / 6)


In [ ]:
# Compute average run rate by batting team
avg_rr = match_summary_df.groupby('batting_team')['run_rate'].mean().reset_index()
avg_rr.rename(columns={'batting_team': 'team', 'run_rate': 'avg_run_rate'}, inplace=True)
# Compute average economy rate by bowling team
avg_er = bowling_summary_df.groupby('bowling_team')['economy_rate'].mean().reset_index()
avg_er.rename(columns={'bowling_team': 'team', 'economy_rate': 'avg_economy_rate'}, inplace=True)
# Merge these two on team
rr_er = pd.merge(avg_rr, avg_er, on='team', how='outer')

 Average Run Rate (Batting) & Economy Rate (Bowling)

In [ ]:


fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=rr_er['team'],
    y=rr_er['avg_run_rate'],
    name='Avg Run Rate',
    marker_color='royalblue',
    text=rr_er['avg_run_rate'].round(2),  # Add text for Avg Run Rate
    textposition='outside'  # Position text outside the bars
))

fig2.add_trace(go.Bar(
    x=rr_er['team'],
    y=rr_er['avg_economy_rate'],
    name='Avg Economy Rate',
    marker_color='indianred',
    text=rr_er['avg_economy_rate'].round(2),  # Add text for Avg Economy Rate
    textposition='outside'  # Position text outside the bars
))

fig2.update_layout(
    title='Average Run Rate (Batting) & Economy Rate (Bowling) by Team',
    xaxis_title='Team',
    yaxis_title='Rate',
    barmode='group'
)
fig2.show()

Highest and Lowest Scores by Team (Batting)

In [ ]:
team_scores = match_summary_df.groupby('batting_team').agg(
    highest_score=('total_runs', 'max'),
    lowest_score=('total_runs', 'min')
).reset_index()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=team_scores['batting_team'],
    y=team_scores['highest_score'],
    name='Highest Score',
    mode='markers+lines',
    marker=dict(color='seagreen', size=10)
))
fig3.add_trace(go.Scatter(
    x=team_scores['batting_team'],
    y=team_scores['lowest_score'],
    name='Lowest Score',
    mode='markers+lines',
    marker=dict(color='darkorange', size=10)
))
fig3.update_layout(
    title='Highest and Lowest Scores by Team',
    xaxis_title='Team',
    yaxis_title='Score'
)
fig3.show()



Total 4s and 6s by Team (Batting) using Bar Chart


In [ ]:
boundaries = match_summary_df.groupby('batting_team').agg(
    total_4s=('num_4s', 'sum'),
    total_6s=('num_6s', 'sum')
).reset_index()

fig4 = go.Figure()
fig4.add_trace(go.Bar(
    x=boundaries['batting_team'],
    y=boundaries['total_4s'],
    name='Total 4s',
    marker_color='mediumpurple'
))
fig4.add_trace(go.Bar(
    x=boundaries['batting_team'],
    y=boundaries['total_6s'],
    name='Total 6s',
    marker_color='teal'
))
fig4.update_layout(
    title='Total 4s and 6s by Team',
    xaxis_title='Team',
    yaxis_title='Count',
    barmode='group'
)
fig4.show()

Average Powerplay and Death Over Scores by Team (Batting) using Scatter Plot


In [ ]:

over_scores = match_summary_df.groupby('batting_team').agg(
    avg_powerplay_score=('powerplay_score', 'mean'),
    avg_death_over_score=('death_over_score', 'mean')
).reset_index()

fig5 = go.Figure()
fig5.add_trace(go.Scatter(
    x=over_scores['batting_team'],
    y=over_scores['avg_powerplay_score'],
    name='Avg Powerplay Score',
    mode='markers+lines',
    marker=dict(color='slateblue', size=10)
))
fig5.add_trace(go.Scatter(
    x=over_scores['batting_team'],
    y=over_scores['avg_death_over_score'],
    name='Avg Death Over Score',
    mode='markers+lines',
    marker=dict(color='crimson', size=10)
))
fig5.update_layout(
    title='Average Powerplay and Death Over Scores by Team',
    xaxis_title='Team',
    yaxis_title='Average Score'
)
fig5.show()


Powerplay Analysis: Distribution of Powerplay Scores by Team using Box Plot


In [ ]:
fig6 = go.Figure()
teams = match_summary_df['batting_team'].unique()

for team in teams:
    team_data = match_summary_df[match_summary_df['batting_team'] == team]
    fig6.add_trace(go.Box(
        y=team_data['powerplay_score'],
        name=team,
        boxmean='sd'  # shows mean and standard deviation
    ))

fig6.update_layout(
    title='Distribution of Powerplay Scores by Team',
    xaxis_title='Team',
    yaxis_title='Powerplay Score'
)
fig6.show()


BATSMAN DATAFRAME FOR PLAYER ANALYSIS

In [ ]:
batsman_df = deliveries_df.groupby(['match_id', 'batter'])['batsman_runs'].agg(['sum'])
batsman_df = batsman_df.rename(columns={'sum': 'total_runs'})

# Add columns for 6s, 4s, 2s, and 1s
for run_value in [6, 4, 2, 1]:
    batsman_df[f'no_of_{run_value}s'] = deliveries_df.groupby(['match_id', 'batter'])['batsman_runs'].apply(lambda x: (x == run_value).sum()).reset_index(name=f'no_of_{run_value}s').set_index(['match_id', 'batter'])[f'no_of_{run_value}s']

# Add column for dismissal
batsman_df['is_out'] = deliveries_df.groupby(['match_id', 'batter'])['player_dismissed'].apply(lambda x: x.notnull().any()).reset_index(name='is_out').set_index(['match_id', 'batter'])['is_out']

# mapping of match_id to season
match_season_mapping = matches_df.set_index('id')['season'].to_dict()

batsman_df['season'] = batsman_df.index.get_level_values('match_id').map(match_season_mapping)

batsman_df = batsman_df.reset_index().set_index(['season', 'match_id', 'batter'])

batsman_df = batsman_df.reset_index()

TOP 20 SCORERS

In [ ]:


top_20_scorers = batsman_df.groupby('batter')['total_runs'].sum().sort_values(ascending=False).head(20)

# Create a bar plot using Plotly Express
fig = px.bar(
    x=top_20_scorers.index,  # Batsmen names on the x-axis
    y=top_20_scorers.values,  # Total runs on the y-axis
    labels={'x': 'Batsman', 'y': 'Total Runs'},  # Axis labels
    title='Top 20 Run Scorers in IPL'  # Chart title
)

# Customize the layout (optional)
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    xaxis_title_font={'size': 14},  # Increase font size of x-axis title
    yaxis_title_font={'size': 14},  # Increase font size of y-axis title
    title_font={'size': 16}  # Increase font size of chart title
)

# Show the plot
fig.show()

Find Highest Average and Strike Rate for players with >50 matches


In [ ]:
# Calculate total runs and dismissals for each batsman
# For batting average we counted no of dissmissals to abide by official rules
batsman_stats = batsman_df.groupby('batter').agg(
    total_runs=('total_runs', 'sum'),
    dismissals=('is_out', 'sum'),
    balls_faced=('total_runs', 'count')  # Assuming each row represents a ball faced
)

# Calculate batting average and strike rate
batsman_stats['batting_average'] = batsman_stats['total_runs'] / batsman_stats['dismissals']
batsman_stats['strike_rate'] = (batsman_stats['total_runs'] / batsman_stats['balls_faced']) * 100

# Filter for top 20 run-scorers
top_20_stats = batsman_stats.loc[top_20_scorers.index]




```
```

Total runs for top-20 batsman

In [ ]:

# Create a bar chart for total runs
fig_runs = go.Figure(data=[go.Bar(x=top_20_stats.index, y=top_20_stats['total_runs'])])
fig_runs.update_layout(title_text='Total Runs for Top 20 Batsmen', xaxis_title='Batsman', yaxis_title='Total Runs')

fig_runs.show()


Total dissmissals for top 20 bowlers

In [ ]:

# Create a bar chart for dismissals
fig_dismissals = go.Figure(data=[go.Bar(x=top_20_stats.index, y=top_20_stats['dismissals'])])
fig_dismissals.update_layout(title_text='Dismissals for Top 20 Batsmen', xaxis_title='Batsman', yaxis_title='Dismissals')
fig_dismissals.show()



batting average vs. strike rate for top-20 batsmen


In [ ]:
fig_avg_sr = go.Figure(data=[go.Scatter(
    x=top_20_stats['batting_average'],
    y=top_20_stats['strike_rate'],
    mode='markers+text',  # Added 'text' to mode
    text=top_20_stats.index,  # Player names for text labels
    textposition='top center',  # Position of text labels
    customdata=top_20_stats[['batting_average', 'strike_rate']],  # Custom data for hover
    hovertemplate='<b>%{text}</b><br>Avg: %{customdata[0]:.2f}<br>SR: %{customdata[1]:.2f}<extra></extra>'  # Hover template
)])
fig_avg_sr.update_layout(
    title_text='Batting Average vs. Strike Rate for Top 20 Batsmen',
    xaxis_title='Batting Average',
    yaxis_title='Strike Rate'
)
fig_avg_sr.show()

In [ ]:
# Group by match_id and bowler
bowler_df = deliveries_df.groupby(['match_id', 'bowler'])['total_runs'].agg(['sum'])
bowler_df = bowler_df.rename(columns={'sum': 'total_runs_conceded'})

# columns for wickets (bowled, caught, lbw, stumped, hit wicket)
for dismissal_type in ['bowled', 'caught', 'lbw', 'stumped', 'hit wicket']:
    bowler_df[dismissal_type] = deliveries_df.groupby(['match_id', 'bowler'])['dismissal_kind'].apply(lambda x: (x == dismissal_type).sum()).reset_index(name=dismissal_type).set_index(['match_id', 'bowler'])[dismissal_type]

# column for total dismissals (sum of all wicket types)
bowler_df['total_dismissals'] = bowler_df[['bowled', 'caught', 'lbw', 'stumped', 'hit wicket']].sum(axis=1)

#  column for balls bowled (assuming each row represents a ball bowled)
bowler_df['balls_bowled'] = deliveries_df.groupby(['match_id', 'bowler'])['total_runs'].count().reset_index(name='balls_bowled').set_index(['match_id', 'bowler'])['balls_bowled']

#  economy rate (runs conceded per over)
bowler_df['economy'] = (bowler_df['total_runs_conceded'] / bowler_df['balls_bowled']) * 6

bowler_df['season'] = bowler_df.index.get_level_values('match_id').map(match_season_mapping)

bowler_df = bowler_df.reset_index().set_index(['season', 'match_id', 'bowler'])

bowler_df = bowler_df.reset_index()

# Display the resulting DataFrame
fig = go.Figure(data=[go.Table(
    header=dict(values=list(top_20_stats.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[top_20_stats.index, top_20_stats['total_runs'], top_20_stats['dismissals'], top_20_stats['batting_average'], top_20_stats['strike_rate']],
               fill_color='lavender',
               align='left'))
])

fig.show()

Plot top wicket-takers


In [ ]:

# Group by bowler and sum total dismissals
top_wicket_takers = bowler_df.groupby('bowler')['total_dismissals'].sum().sort_values(ascending=False).head(10)

# Create a bar plot using Plotly Express
fig = px.bar(
    x=top_wicket_takers.index,
    y=top_wicket_takers.values,
    labels={'x': 'Bowler', 'y': 'Total Wickets'},
    title='Top 10 Wicket-Takers'
)

# Customize the layout (optional)
fig.update_layout(
    xaxis_tickangle=-45,  # Rotate x-axis labels
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

# Show the plot
fig.show()

plotting highest batting and bowling scores

In [ ]:


#  top 10 highest batting scores
top_batting_scores = batsman_df.groupby('batter')['total_runs'].max().sort_values(ascending=False).head(10)

#  top 10 highest bowling wickets
top_bowling_wickets = bowler_df.groupby('bowler')['total_dismissals'].max().sort_values(ascending=False).head(10)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=top_batting_scores.index,
        y=top_batting_scores.values,
        name='Highest Batting Score',
        mode='lines+markers',  # Display lines and markers
        marker_color='blue'
    )
)

fig.add_trace(
    go.Scatter(
        x=top_bowling_wickets.index,
        y=top_bowling_wickets.values,
        name='Highest Bowling Wickets',
        mode='lines+markers',  # Display lines and markers
        marker_color='red',
        yaxis='y2'  # Assign to secondary y-axis
    )
)

fig.update_layout(
    title='Top Highest Individual Scores',
    xaxis_title='Player',
    yaxis_title='Highest Batting Score',
    yaxis2=dict(
        title='Highest Bowling Wickets',
        overlaying='y',
        side='right'
    )
)

# Show the plot
fig.show()

Man of the Match Count Analysis


In [ ]:

# Group by player_of_match and count the number of awards
man_of_match_counts = matches_df['player_of_match'].value_counts().head(10)

# Create a bar plot.
fig = px.bar(
    x=man_of_match_counts.index,
    y=man_of_match_counts.values,
    labels={'x': 'Player', 'y': 'Number of Man of the Match Awards'},
    title='Top 10 Man of the Match Award Winners'
)

# Customize the layout (optional)
fig.update_layout(
    xaxis_tickangle=-45,
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

# Show the plot
fig.show()

In [ ]:
# batting average
batsman_stats = batsman_df.groupby('batter').agg(
    total_runs=('total_runs', 'sum'),
    dismissals=('is_out', 'sum'),
)
batsman_stats['batting_average'] = batsman_stats['total_runs'] / batsman_stats['dismissals']
batsman_stats = batsman_stats[batsman_stats['dismissals'] >= 10]  # Filter for minimum dismissals

# bowling economy rate
bowler_stats = bowler_df.groupby('bowler').agg(
    total_runs_conceded=('total_runs_conceded', 'sum'),
    balls_bowled=('balls_bowled', 'sum'),
)
bowler_stats['economy'] = (bowler_stats['total_runs_conceded'] / bowler_stats['balls_bowled']) * 6
bowler_stats = bowler_stats[bowler_stats['balls_bowled'] >= 120]  # Filter for minimum balls bowled

# Merge dataframes on player name
player_stats = pd.merge(batsman_stats, bowler_stats, left_index=True, right_index=True, how='inner')

#  features for clustering
X = player_stats[['batting_average', 'economy']].values

# Performing K-Means clustering
# k=3 (bowler,batter,all-rounder)
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(X)
labels = kmeans.labels_

# Create a DataFrame for plotting
plot_df = pd.DataFrame({'Batting Average': X[:, 0], 'Bowling Economy Rate': X[:, 1], 'Cluster': labels})

# ... (your existing code) ...

# Create the scatter plot using Plotly Express
fig = px.scatter(
    plot_df,
    x='Batting Average',
    y='Bowling Economy Rate',
    color='Cluster',
    title='Batting Average vs Bowling Economy Rate with Clusters',
)

# Update layout to display text labels
fig.update_traces(textposition='top center')

# Show the plot
fig.show()

In [ ]:


# top 10 batsmen for a specific run category
def get_top_10_batsmen(run_category):
    top_batsmen = batsman_df.groupby('batter')[f'no_of_{run_category}s'].sum().sort_values(ascending=False).head(10)
    return top_batsmen

# Get top 10 batsmen for each run category
top_6s_scorers = get_top_10_batsmen(6)
top_4s_scorers = get_top_10_batsmen(4)
top_2s_scorers = get_top_10_batsmen(2)
top_1s_scorers = get_top_10_batsmen(1)

# Create a list of run categories and corresponding top batsmen
run_categories = ['6s', '4s', '2s', '1s']
top_batsmen_data = [top_6s_scorers, top_4s_scorers, top_2s_scorers, top_1s_scorers]

# Create a separate bar plot for each run category
for i, run_category in enumerate(run_categories):
    fig = px.bar(
        x=top_batsmen_data[i].index,
        y=top_batsmen_data[i].values,
        labels={'x': 'Batsman', 'y': f'Number of {run_category}'},
        title=f'Top 10 Batsmen for {run_category}'
    )
    fig.update_layout(
        xaxis_tickangle=-45,  # Rotate x-axis labels
        xaxis_title_font={'size': 14},
        yaxis_title_font={'size': 14},
        title_font={'size': 16}
    )
    fig.show()

Season Analysis


In [ ]:
# Group by season and match_id, then sum total runs
season_match_runs = batsman_df.groupby(['season', 'match_id'])['total_runs'].sum().reset_index()

# Group by season and calculate the average runs per match
avg_runs_per_season = season_match_runs.groupby('season')['total_runs'].mean().reset_index()

# Rename the column for better readability
avg_runs_per_season.rename(columns={'total_runs': 'avg_runs_per_match'}, inplace=True)

# Print or display the result
avg_runs_per_season


# Assuming avg_runs_per_season is already calculated

# Create a line plot using Plotly Express
fig = px.line(
    avg_runs_per_season,
    x='season',
    y='avg_runs_per_match',
    labels={'season': 'Season', 'avg_runs_per_match': 'Average Runs per Match'},
    title='Average Runs per Match per Season'
)

# Customize the layout (optional)
fig.update_layout(
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

# Show the plot
fig.show()

targets of 200+ runs per season

In [ ]:
season_match_runs = batsman_df.groupby(['season', 'match_id'])['total_runs'].sum().reset_index()

In [ ]:
targets_200_plus = season_match_runs[season_match_runs['total_runs'] >= 200]

In [ ]:

# Group by season and count the number of matches with 200+ targets
targets_per_season = targets_200_plus.groupby('season')['match_id'].count().reset_index()

targets_per_season.rename(columns={'match_id': 'target_count'}, inplace=True)

fig = go.Figure(data=go.Scatter(x=targets_per_season['season'], y=targets_per_season['target_count'], mode='lines+markers'))  # Use 'lines+markers' for line chart with markers

fig.update_layout(
    title='Number of 200+ Targets per Season',
    xaxis_title='Season',
    yaxis_title='Number of 200+ Targets',
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

fig.show()

Number of 200+ Targets per Season by Teams


In [ ]:
# Group by season and team (team batting first) and count matches with 200+ runs
team_targets_200_plus = matches_df.merge(targets_200_plus, left_on='id', right_on='match_id', how='inner')

team_targets_200_plus = team_targets_200_plus.groupby(['season_x', 'team1'])['match_id'].count().reset_index()
team_targets_200_plus = team_targets_200_plus.rename(columns={'match_id': 'count', 'season_x':'season'})




In [ ]:
traces = []
for team in team_targets_200_plus['team1'].unique():
    team_data = team_targets_200_plus[team_targets_200_plus['team1'] == team]
    traces.append(
        go.Bar(
            x=team_data['season'],
            y=team_data['count'],
            name=team,
        )
    )

fig = go.Figure(data=traces)

fig.update_layout(
    title='Number of 200+ Targets per Season by Teams',
    xaxis_title='Season',
    yaxis_title='Number of 200+ Targets',
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16},
    barmode='group',
    legend_title_text='Teams',
)

fig.show()

Average Score of Each Team vs Year

In [ ]:
# Merge deliveries_df with matches_df to get team and season information
merged_df = pd.merge(deliveries_df, matches_df, left_on='match_id', right_on='id', how='inner')

team_season_total_runs = merged_df.groupby(['season', 'batting_team'])['total_runs'].sum().reset_index()

team_season_avg_score = team_season_total_runs.groupby(['season', 'batting_team'])['total_runs'].mean().reset_index()

team_season_avg_score.rename(columns={'total_runs': 'average_score'}, inplace=True)
team_season_avg_score = team_season_avg_score.sort_values(by=['season'], ascending=[True])



In [ ]:
type(team_season_avg_score)

pandas.core.frame.DataFrame

In [ ]:
fig = px.line(
    team_season_avg_score,
    x='season',
    y='average_score',
    color='batting_team',
    labels={'season': 'Season', 'average_score': 'Average Score', 'batting_team': 'Team'},
    title='Average Score of Each Team vs Year'
)

fig.update_layout(
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

fig.show()

Orange Cap Holders per Season

In [ ]:
# Group by season and winner, then count the occurrences
orange_cap_holders_per_season = matches_df.groupby(['season', 'winner'])['winner'].count().reset_index(name='count')

max_wins_per_season = orange_cap_holders_per_season.loc[orange_cap_holders_per_season.groupby('season')['count'].idxmax()]

fig = px.bar(
    max_wins_per_season,
    x='season',
    y='count',
    color='winner',
    labels={'season': 'Season', 'count': 'Number of Wins', 'winner': 'Orange Cap Holder'},
    title='Orange Cap Holders per Season'
)

fig.update_layout(
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

fig.show()


Purple cap holder each season

In [ ]:
# Group by season and bowler, then count the occurrences
purple_cap_holders_per_season = bowler_df.groupby(['season', 'bowler'])['total_dismissals'].sum().reset_index(name='total_wickets')

max_wickets_per_season = purple_cap_holders_per_season.loc[purple_cap_holders_per_season.groupby('season')['total_wickets'].idxmax()]

fig = px.bar(
    max_wickets_per_season,
    x='season',
    y='total_wickets',
    color='bowler',
    labels={'season': 'Season', 'total_wickets': 'Number of Wickets', 'bowler': 'Purple Cap Holder'},
    title='Purple Cap Holders per Season'
)

fig.update_layout(
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

fig.show()


Bowlers dataframe

In [ ]:
def get_top_10_bowlers_per_season(df):
  top_bowlers_per_season = df.groupby(['season', 'bowler'])['total_dismissals'].sum().reset_index()
  top_bowlers_per_season = top_bowlers_per_season.groupby('season').apply(lambda x: x.nlargest(10, 'total_dismissals')).reset_index(drop=True)
  return top_bowlers_per_season


top_bowlers_df = get_top_10_bowlers_per_season(bowler_df)


<ipython-input-101-2566c186fbd7>:3: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Bowlers Taking 10+ Wickets per Season

In [ ]:
# Group by season and bowler, then count the occurrences
wickets_per_season = bowler_df.groupby(['season', 'bowler'])['total_dismissals'].sum().reset_index()

# Filter for bowlers with 10 or more wickets in a season
bowlers_10_wickets = wickets_per_season[wickets_per_season['total_dismissals'] >= 10]

# Create a line plot using Plotly Express
fig = px.line(
    bowlers_10_wickets,
    x='season',
    y='total_dismissals',
    color='bowler',  # Color lines by bowler
    labels={'season': 'Season', 'total_dismissals': 'Wickets', 'bowler': 'Bowler'},
    title='Bowlers Taking 10+ Wickets per Season'
)

# Customize the layout (optional)
fig.update_layout(
    xaxis_title_font={'size': 14},
    yaxis_title_font={'size': 14},
    title_font={'size': 16}
)

# Show the plot
fig.show()


In [ ]:

class MatchDataCleaner:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = pd.read_csv(file_path)

    def clean_venues(self):
        venue_mapping = {
            'M.Chinnaswamy Stadium': 'M Chinnaswamy Stadium',
            'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association Stadium, Mohali',
            'Sawai Mansingh Stadium, Jaipur': 'Sawai Mansingh Stadium',
            'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
            'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 'Rajiv Gandhi International Stadium',
            'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
            'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
            'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
            'Brabourne Stadium, Mumbai': 'Brabourne Stadium',
            'Himachal Pradesh Cricket Association Stadium, Dharamsala': 'Himachal Pradesh Cricket Association Stadium',
            'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
            'Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh': 'Punjab Cricket Association IS Bindra Stadium',
            'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
            'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
            'Narendra Modi Stadium, Ahmedabad': 'Narendra Modi Stadium',
            'Zayed Cricket Stadium, Abu Dhabi': 'Sheikh Zayed Stadium',
            'Eden Gardens, Kolkata': 'Eden Gardens',
            'M Chinnaswamy Stadium, Bengaluru': 'M Chinnaswamy Stadium',
            'Barsapara Cricket Stadium, Guwahati': 'Barsapara Cricket Stadium',
            'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur': 'Maharaja Yadavindra Singh Stadium'
        }
        self.df["venue"].replace(venue_mapping, inplace=True)

    def rename_teams(self):
        """Rename old team names to their new equivalents."""
        team_replacements = {
            'Delhi Daredevils': 'Delhi Capitals',
            'Deccan Chargers': 'Sunrisers Hyderabad',
            'Royal Challengers Bengaluru': 'Royal Challengers Bangalore',
            'Punjab Kings': 'Kings XI Punjab',
            'Rising Pune Supergiants': 'Rising Pune Supergiant'
        }
        self.df.replace(team_replacements, inplace=True)

    def drop_teams(self, teams_to_drop):
        """Drop rows where team1 or team2 belongs to the specified teams."""
        self.df = self.df[~self.df['team1'].isin(teams_to_drop) & ~self.df['team2'].isin(teams_to_drop)]

    def clean_season_column(self):
        """Standardize season values."""
        season_mapping = {'2007/08': '2008', '2009/10': '2010', '2020/21': '2020'}
        self.df['season'].replace(season_mapping, inplace=True)

    def get_cleaned_data(self):
        """Return the cleaned DataFrame."""
        return self.df


In [ ]:
class IPLDataProcessor:
    def __init__(self, matches_df: pd.DataFrame, deliveries_df: pd.DataFrame):
        """
        Initialize with the raw matches and deliveries data.
        """
        self.matches_df = matches_df.copy()
        self.deliveries_df = deliveries_df.copy()

    def make_base_features(self):
        """
        Compute overall team-level features using the entire matches and deliveries datasets.
        Returns a DataFrame with one row per team.
        Features:
            - Win_Rate
            - Avg_Score (Average match score)
            - Avg_Conceded (Average runs conceded)
            - Toss_Win_Rate
        """
        # 1. Compute total matches played per team.
        teams = pd.concat([
            self.matches_df[['team1']].rename(columns={'team1': 'team'}),
            self.matches_df[['team2']].rename(columns={'team2': 'team'})
        ])
        teams_played = teams.groupby('team').size().reset_index(name='matches_played')

        # 2. Compute wins and win rate.
        wins = self.matches_df['winner'].value_counts().reset_index()
        wins.columns = ['team', 'wins']
        win_rate_df = pd.merge(teams_played, wins, on='team', how='left')
        win_rate_df['wins'] = win_rate_df['wins'].fillna(0)
        win_rate_df['Win_Rate'] = win_rate_df['wins'] / win_rate_df['matches_played']

        # 3. Compute average match score (runs scored when batting).
        match_scores = self.deliveries_df.groupby(['match_id', 'batting_team'])['total_runs'] \
                                        .sum().reset_index()
        match_scores.rename(columns={'batting_team': 'team', 'total_runs': 'match_score'}, inplace=True)
        avg_score = match_scores.groupby('team')['match_score'].mean().reset_index()
        avg_score.rename(columns={'match_score': 'Avg_Score'}, inplace=True)

        # 4. Compute average runs conceded (by bowling).
        conceded_scores = self.deliveries_df.groupby(['match_id', 'bowling_team'])['total_runs'] \
                                            .sum().reset_index()
        conceded_scores.rename(columns={'bowling_team': 'team', 'total_runs': 'conceded_score'}, inplace=True)
        avg_conceded = conceded_scores.groupby('team')['conceded_score'].mean().reset_index()
        avg_conceded.rename(columns={'conceded_score': 'Avg_Conceded'}, inplace=True)

        # 5. Compute Toss Win Rate.
        toss_df = self.matches_df.copy()
        toss_df['toss_win_match'] = (toss_df['toss_winner'] == toss_df['winner']).astype(int)
        toss_stats = toss_df.groupby('toss_winner')['toss_win_match'].agg(['sum', 'count']).reset_index()
        toss_stats.rename(columns={'toss_winner': 'team', 'sum': 'toss_wins_conversion', 'count': 'toss_total'}, inplace=True)
        toss_stats['Toss_Win_Rate'] = toss_stats['toss_wins_conversion'] / toss_stats['toss_total']

        # 6. Merge all features together.
        base_features = pd.merge(win_rate_df[['team', 'Win_Rate']], avg_score, on='team', how='left')
        base_features = pd.merge(base_features, avg_conceded, on='team', how='left')
        base_features = pd.merge(base_features, toss_stats[['team', 'Toss_Win_Rate']], on='team', how='left')

        return base_features

    def make_head2head_features(self):
        """
        Compute head-to-head features.

        For each team, calculates the average head-to-head win rate
        against all opponents based on historical match-ups.
        """
        records = []
        for idx, row in self.matches_df.iterrows():
            team1 = row['team1']
            team2 = row['team2']
            winner = row['winner']
            # Only consider matches with a valid winner.
            if pd.notnull(winner):
                records.append({'team': team1, 'opponent': team2, 'win': 1 if winner == team1 else 0})
                records.append({'team': team2, 'opponent': team1, 'win': 1 if winner == team2 else 0})
        h2h_df = pd.DataFrame(records)
        if h2h_df.empty:
            return pd.DataFrame(columns=['team', 'H2H_Win_Rate'])
        h2h_summary = h2h_df.groupby(['team', 'opponent'])['win'].agg(['sum', 'count']).reset_index()
        h2h_summary['h2h_win_rate'] = h2h_summary['sum'] / h2h_summary['count']
        return h2h_summary

    def make_seasonal_features(self):
        """
        Generate seasonal features per team.

        For each season, calculates:
            - Season Win Rate.
            - Season Average Score.
            - Season Average Runs Conceded.
            - Season Toss Win Conversion Rate.

        Returns a DataFrame with each row representing a (team, season) pair.
        """
        seasonal_features = []
        # Ensure the matches_df has a 'season' column.
        if 'season' not in self.matches_df.columns:
            raise ValueError("matches_df must have a 'season' column for seasonal features.")

        seasons = self.matches_df['season'].unique()
        for season in seasons:
            season_matches = self.matches_df[self.matches_df['season'] == season]

            # --- Season Win Rate ---
            matches_played = pd.concat([
                season_matches[['team1']].rename(columns={'team1': 'team'}),
                season_matches[['team2']].rename(columns={'team2': 'team'})
            ])
            matches_played = matches_played.groupby('team').size().reset_index(name='matches_played')

            # Use groupby instead of value_counts to ensure the column is named 'team'
            wins = season_matches.groupby('winner').size().reset_index(name='wins').rename(columns={'winner': 'team'})

            win_rate_df = pd.merge(matches_played, wins, on='team', how='left')
            win_rate_df['wins'].fillna(0, inplace=True)
            win_rate_df['Season_Win_Rate'] = win_rate_df['wins'] / win_rate_df['matches_played']
            win_rate_df['season'] = season

            # --- Season Average Score ---
            # Filter deliveries for the current season using match IDs.
            season_deliveries = self.deliveries_df[self.deliveries_df['match_id'].isin(season_matches['id'])]
            match_scores = season_deliveries.groupby(['match_id', 'batting_team'])['total_runs']\
                                            .sum().reset_index().rename(columns={'batting_team': 'team',
                                                                                  'total_runs': 'match_score'})
            season_avg_score = match_scores.groupby('team')['match_score'].mean()\
                                          .reset_index().rename(columns={'match_score': 'Season_Avg_Score'})

            # --- Season Average Runs Conceded ---
            conceded_scores = season_deliveries.groupby(['match_id', 'bowling_team'])['total_runs']\
                                              .sum().reset_index().rename(columns={'bowling_team': 'team',
                                                                                    'total_runs': 'conceded_score'})
            season_avg_conceded = conceded_scores.groupby('team')['conceded_score'].mean()\
                                                  .reset_index().rename(columns={'conceded_score': 'Season_Avg_Conceded'})

            # --- Season Toss Win Conversion Rate ---
            toss_df = season_matches[pd.notnull(season_matches['toss_winner'])].copy()
            toss_df['toss_win_match'] = (toss_df['toss_winner'] == toss_df['winner']).astype(int)
            toss_stats = toss_df.groupby('toss_winner').agg({'toss_win_match': ['sum', 'count']})
            toss_stats.columns = ['toss_wins_conversion', 'toss_total']
            toss_stats = toss_stats.reset_index().rename(columns={'toss_winner': 'team'})
            toss_stats['Season_Toss_Win_Rate'] = toss_stats['toss_wins_conversion'] / toss_stats['toss_total']

            # Merge seasonal features for this season
            season_features = pd.merge(win_rate_df[['team', 'Season_Win_Rate', 'season']],
                                      season_avg_score, on='team', how='left')
            season_features = pd.merge(season_features, season_avg_conceded, on='team', how='left')
            season_features = pd.merge(season_features, toss_stats[['team', 'Season_Toss_Win_Rate']], on='team', how='left')
            seasonal_features.append(season_features)

        seasonal_features_df = pd.concat(seasonal_features, ignore_index=True)
        return seasonal_features_df

    def make_features(self):
        """
        Combine base, seasonal, and head-to-head features with the matches_df so that
        each match (row) gets features for both team1 and team2.

        For team-level features (base and seasonal), the function merges features for team1 and team2
        separately (with prefixes) based on the team name and season.

        For head-to-head features, it looks up the historical head-to-head win rate for team1 against team2
        and vice versa.

        Returns:
            final_df: A DataFrame with one row per match (indexed by match_id) and all the combined features.
        """
        # 1. Get all feature sets.
        base_features = self.make_base_features()          # overall team-level features; one row per team.
        seasonal_features = self.make_seasonal_features()    # seasonal (team, season) features.
        h2h_features = self.make_head2head_features()         # head-to-head features: (team, opponent) pairs.

        # 2. Start with the original matches DataFrame.
        #    Rename the match id column (if necessary) to 'match_id'
        final_df = self.matches_df.copy()
        if 'id' in final_df.columns:
            final_df.rename(columns={'id': 'match_id'}, inplace=True)

        # 3. Merge base features for team1.
        final_df = final_df.merge(base_features, left_on='team1', right_on='team', how='left') \
                          .rename(columns={
                              'Win_Rate': 'team1_Win_Rate',
                              'Avg_Score': 'team1_Avg_Score',
                              'Avg_Conceded': 'team1_Avg_Conceded',
                              'Toss_Win_Rate': 'team1_Toss_Win_Rate'
                          }).drop(columns=['team'])

        # Merge base features for team2.
        final_df = final_df.merge(base_features, left_on='team2', right_on='team', how='left') \
                          .rename(columns={
                              'Win_Rate': 'team2_Win_Rate',
                              'Avg_Score': 'team2_Avg_Score',
                              'Avg_Conceded': 'team2_Avg_Conceded',
                              'Toss_Win_Rate': 'team2_Toss_Win_Rate'
                          }).drop(columns=['team'])

        # 4. Merge seasonal features for team1 using (team1, season).
        final_df = final_df.merge(seasonal_features, left_on=['team1', 'season'],
                                  right_on=['team', 'season'], how='left') \
                          .rename(columns={
                              'Season_Win_Rate': 'team1_Season_Win_Rate',
                              'Season_Avg_Score': 'team1_Season_Avg_Score',
                              'Season_Avg_Conceded': 'team1_Season_Avg_Conceded',
                              'Season_Toss_Win_Rate': 'team1_Season_Toss_Win_Rate'
                          }).drop(columns=['team'])

        # Merge seasonal features for team2 using (team2, season).
        final_df = final_df.merge(seasonal_features, left_on=['team2', 'season'],
                                  right_on=['team', 'season'], how='left') \
                          .rename(columns={
                              'Season_Win_Rate': 'team2_Season_Win_Rate',
                              'Season_Avg_Score': 'team2_Season_Avg_Score',
                              'Season_Avg_Conceded': 'team2_Season_Avg_Conceded',
                              'Season_Toss_Win_Rate': 'team2_Season_Toss_Win_Rate'
                          }).drop(columns=['team'])

        # 5. Add head-to-head features.
        # Create a lookup dictionary from the head-to-head DataFrame.
        h2h_lookup = h2h_features.set_index(['team', 'opponent'])['h2h_win_rate'].to_dict()

        # Define a helper function to get the head-to-head win rate.
        def get_h2h_rate(team, opponent):
            return h2h_lookup.get((team, opponent), None)

        # Apply the lookup for team1 vs team2 and for team2 vs team1.
        final_df['team1_H2H_vs_team2'] = final_df.apply(lambda row: get_h2h_rate(row['team1'], row['team2']), axis=1)
        final_df['team2_H2H_vs_team1'] = final_df.apply(lambda row: get_h2h_rate(row['team2'], row['team1']), axis=1)

        # 6. Set match_id as the index (if desired) and return the final dataset.
        final_df.set_index('match_id', inplace=True)
        return final_df


In [ ]:
data_processor = IPLDataProcessor(matches_df, deliveries_df)
base_features = data_processor.make_base_features()
h2h_features = data_processor.make_head2head_features()
seasonal_features = data_processor.make_seasonal_features()
final_df = data_processor.make_features()

<ipython-input-104-18c13a6d74ee>:116: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



<ipython-input-104-18c13a6d74ee>:116: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' 

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1095 entries, 335982 to 1426312
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   season                      1095 non-null   object 
 1   city                        1044 non-null   object 
 2   date                        1095 non-null   object 
 3   match_type                  1095 non-null   object 
 4   player_of_match             1090 non-null   object 
 5   venue                       1095 non-null   object 
 6   team1                       1095 non-null   object 
 7   team2                       1095 non-null   object 
 8   toss_winner                 1095 non-null   object 
 9   toss_decision               1095 non-null   object 
 10  winner                      1090 non-null   object 
 11  result                      1095 non-null   object 
 12  result_margin               1076 non-null   float64
 13  target_runs                 10

In [ ]:
final_df.to_csv('matches_with_features.csv',index=False)

In [ ]:
cleaner = MatchDataCleaner("matches_with_features.csv")
cleaner.clean_venues()
cleaner.rename_teams()
cleaner.drop_teams(['Kochi Tuskers Kerala', 'Pune Warriors', 'Gujarat Lions', 'Rising Pune Supergiant'])
cleaner.clean_season_column()
df2 = cleaner.get_cleaned_data()

<ipython-input-103-30ce20831ea0>:32: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



<ipython-input-103-30ce20831ea0>:52: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or

In [ ]:
df2['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab', 'Delhi Capitals',
       'Mumbai Indians', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Sunrisers Hyderabad', 'Chennai Super Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [ ]:
df2['team1'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab', 'Delhi Capitals',
       'Mumbai Indians', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Sunrisers Hyderabad', 'Chennai Super Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [ ]:
df2.shape

(980, 37)

In [ ]:
df2.tail()

,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,...,team1_Season_Win_Rate,team1_Season_Avg_Score,team1_Season_Avg_Conceded,team1_Season_Toss_Win_Rate,team2_Season_Win_Rate,team2_Season_Avg_Score,team2_Season_Avg_Conceded,team2_Season_Toss_Win_Rate,team1_H2H_vs_team2,team2_H2H_vs_team1
1090,2024,Hyderabad,2024-05-19,League,Abhishek Sharma,Rajiv Gandhi International Stadium,Kings XI Punjab,Sunrisers Hyderabad,Kings XI Punjab,bat,...,0.357143,177.642857,186.571429,0.400000,0.562500,190.750000,188.500000,0.285714,0.424242,0.575758
1091,2024,Ahmedabad,2024-05-21,Qualifier 1,MA Starc,Narendra Modi Stadium,Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,...,0.562500,190.750000,188.500000,0.285714,0.785714,190.500000,171.928571,1.000000,0.297297,0.702703
1092,2024,Ahmedabad,2024-05-22,Eliminator,R Ashwin,Narendra Modi Stadium,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,...,0.466667,195.333333,188.000000,0.500000,0.600000,176.466667,177.133333,0.636364,0.517241,0.482759
1093,2024,Chennai,2024-05-24,Qualifier 2,Shahbaz Ahmed,MA Chidambaram Stadium,Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals,field,...,0.562500,190.750000,188.500000,0.285714,0.600000,176.466667,177.133333,0.636364,0.448276,0.551724
1094,2024,Chennai,2024-05-26,Final,MA Starc,MA Chidambaram Stadium,Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,...,0.562500,190.750000,188.500000,0.285714,0.785714,190.500000,171.928571,1.000000,0.297297,0.702703


In [ ]:
df2['team2'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Royal Challengers Bangalore', 'Sunrisers Hyderabad',
       'Kings XI Punjab', 'Delhi Capitals', 'Mumbai Indians',
       'Gujarat Titans', 'Lucknow Super Giants'], dtype=object)

In [ ]:
df2.head()

,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,...,team1_Season_Win_Rate,team1_Season_Avg_Score,team1_Season_Avg_Conceded,team1_Season_Toss_Win_Rate,team2_Season_Win_Rate,team2_Season_Avg_Score,team2_Season_Avg_Conceded,team2_Season_Toss_Win_Rate,team1_H2H_vs_team2,team2_H2H_vs_team1
0,2008,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,...,0.285714,141.642857,157.500000,0.200000,0.461538,149.384615,132.153846,0.500000,0.411765,0.588235
1,2008,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,...,0.666667,164.266667,161.133333,0.500000,0.562500,157.500000,160.500000,0.600000,0.466667,0.533333
2,2008,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,bat,...,0.500000,151.285714,158.785714,0.333333,0.812500,162.562500,150.187500,0.818182,0.482759,0.517241
3,2008,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,...,0.500000,148.571429,149.714286,0.500000,0.285714,141.642857,157.500000,0.200000,0.575758,0.424242
4,2008,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,bat,...,0.461538,149.384615,132.153846,0.500000,0.142857,159.214286,164.785714,0.222222,0.702703,0.297297


In [ ]:
df2['venue'].nunique()

38

In [ ]:


class makeTrainDataset:
    def __init__(self, df):
        self.df = df.copy()  # Avoid modifying the original DataFrame
        self.drop_columns()  # Drop unnecessary columns
        self.df['season'] = pd.to_numeric(self.df['season'], errors='coerce')  # Convert season to numeric
        self.encoders = {}  # Dictionary to store LabelEncoders
        self.team_mapping = {}  # Initialize team_mapping to prevent attribute errors

        # Encode team-related columns separately
        self.encode_teams()

    def drop_columns(self):
        """Drops unnecessary columns from the dataset."""
        columns_to_drop = [
            'id', 'city', 'date', 'player_of_match', 'umpire1', 'umpire2',
            'result', 'result_margin', 'super_over', 'method', 'target_overs', 'target_runs'
        ]
        self.df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    def encode_column(self, column_name):
        """Encodes a given column using Label Encoding and updates the DataFrame."""
        team_columns = ["team1", "team2", "toss_winner", "winner"]

        if column_name in team_columns:
            print(f"Skipping encoding for '{column_name}' as teams are encoded together.")
            return {}

        if column_name not in self.df.columns:
            print(f"Column '{column_name}' not found in the DataFrame.")
            return {}

        self.df.dropna(subset=[column_name], inplace=True)

        # Create and fit LabelEncoder for the column
        encoder = LabelEncoder()
        self.df[column_name] = encoder.fit_transform(self.df[column_name])

        # Store encoder for future reference
        self.encoders[column_name] = encoder

        # Return the mapping as a dictionary
        return dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

    def encode_teams(self):
        """Encodes team-related columns using a single LabelEncoder for consistency."""
        team_columns = ["team1", "team2", "toss_winner", "winner"]

        # Drop rows with missing team values
        self.df.dropna(subset=team_columns, inplace=True)

        # Gather all unique team names
        unique_teams = pd.concat([self.df[col] for col in team_columns]).unique()

        # Create a single encoder for teams
        team_encoder = LabelEncoder()
        team_encoder.fit(unique_teams)

        # Store encoder
        self.encoders["teams"] = team_encoder

        # Ensure self.team_mapping is always initialized
        self.team_mapping = dict(zip(team_encoder.classes_, team_encoder.transform(team_encoder.classes_)))

        # Apply encoding to all team columns
        for col in team_columns:
            self.df[col] = team_encoder.transform(self.df[col])

    def split_data(self, season):
        """Splits the dataset into train, validation, and test sets based on the given season."""
        train = self.df[self.df['season'] <= season]
        val = self.df[(self.df['season'] > season) & (self.df['season'] <= season + 1)]
        test = self.df[self.df['season'] >= season + 1].drop(columns=['winner'], errors='ignore')

        # Separate features and target for train and validation
        X_train = train.drop(columns=['winner'])
        y_train = train['winner']
        X_val = val.drop(columns=['winner'])
        y_val = val['winner']

        return X_train, y_train, X_val, y_val, test




In [ ]:

class makeTrainDataset:
    def __init__(self, df):
        self.df = df  # Directly modify the original DataFrame
        self.drop_columns()
        self.df['season'] = pd.to_numeric(self.df['season'], errors='coerce')
        self.encoders = {}
        self.team_mapping = {}

        self.encode_teams()

    def drop_columns(self):
        """Drops unnecessary columns from the dataset."""
        columns_to_drop = [
            'id', 'city', 'date', 'player_of_match', 'umpire1', 'umpire2',
            'result', 'result_margin', 'super_over', 'method', 'target_overs', 'target_runs'
        ]
        self.df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    def encode_column(self, column_name):
        """Encodes a given column using Label Encoding and updates the DataFrame."""
        team_columns = ["team1", "team2", "toss_winner", "winner"]

        if column_name in team_columns:
            print(f"Skipping encoding for '{column_name}' as teams are encoded together.")
            return {}

        if column_name not in self.df.columns:
            print(f"Column '{column_name}' not found in the DataFrame.")
            return {}

        self.df.dropna(subset=[column_name], inplace=True)

        encoder = LabelEncoder()
        self.df[column_name] = encoder.fit_transform(self.df[column_name])

        self.encoders[column_name] = encoder
        return dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

    def encode_teams(self):
        """Encodes team-related columns using a single LabelEncoder for consistency."""
        team_columns = ["team1", "team2", "toss_winner", "winner"]

        self.df.dropna(subset=team_columns, inplace=True)

        unique_teams = pd.concat([self.df[col] for col in team_columns]).unique()

        team_encoder = LabelEncoder()
        team_encoder.fit(unique_teams)

        self.encoders["teams"] = team_encoder

        self.team_mapping = dict(zip(team_encoder.classes_, team_encoder.transform(team_encoder.classes_)))

        for col in team_columns:
            self.df[col] = team_encoder.transform(self.df[col])

    def split_data(self, season):
        """Splits the dataset and fills missing test values with the last row of X_train."""
        # Split dataset
        train = self.df[self.df['season'] <= season]
        val = self.df[(self.df['season'] > season) & (self.df['season'] <= season + 1)]
        test = self.df[self.df['season'] > season + 1].drop(columns=['winner'], errors='ignore')

        # Separate features and target for train and validation
        X_train = train.drop(columns=['winner'])
        y_train = train['winner']
        X_val = val.drop(columns=['winner'])
        y_val = val['winner']

        # Columns to replace in the test set
        cols_to_replace = [
            'team1_Season_Win_Rate', 'team1_Season_Avg_Score', 'team1_Season_Avg_Conceded',
            'team1_Season_Toss_Win_Rate', 'team2_Season_Win_Rate', 'team2_Season_Avg_Score',
            'team2_Season_Avg_Conceded', 'team2_Season_Toss_Win_Rate', 'team1_H2H_vs_team2',
            'team2_H2H_vs_team1'
        ]

        # Get the last row of X_train
        if not X_train.empty:
            last_train_row = X_train.iloc[-1][cols_to_replace]

            # Assign last row's values to test set
            for col in cols_to_replace:
                if col in test.columns:
                    test[col] = last_train_row[col]

        return X_train, y_train, X_val, y_val, test


In [ ]:
dataset = makeTrainDataset(df2)

# Print team encoding map
# team_map=dataset.encode_teams()
# print(team_map)
dataset.drop_columns()
# Encode another column separately (e.g., venue)
# venue_mapping = dataset.encode_column("venue")
# match_mapping = dataset.encode_column("match_type")
# toss_mapping = dataset.encode_column("toss_decision")
# print(match_mapping)
# print(toss_mapping)
# print(venue_mapping)

X_train, y_train, X_val, y_val, test = dataset.split_data(2022)

NameError: name 'LabelEncoder' is not defined

In [ ]:
df2.head()

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
df2.shape

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score


In [ ]:
# Convert datasets into DMatrix format
X_train['match_type'] = X_train['match_type'].astype('category')
X_train['venue'] = X_train['venue'].astype('category')
X_train['toss_decision'] = X_train['toss_decision'].astype('category')
X_val['match_type'] = X_val['match_type'].astype('category')
X_val['venue'] = X_val['venue'].astype('category')
X_val['toss_decision'] = X_val['toss_decision'].astype('category')
test['match_type'] = test['match_type'].astype('category')
test['venue'] = test['venue'].astype('category')
test['toss_decision'] = test['toss_decision'].astype('category')
dtrain = xgb.DMatrix(X_train, label=y_train,enable_categorical=True)
dval = xgb.DMatrix(X_val, label=y_val,enable_categorical=True)
dtest = xgb.DMatrix(test,enable_categorical=True)  # Test set (no labels)

In [ ]:
params = {
    'objective': 'multi:softmax',  # Multi-class classification
    'num_class': 10,                # Adjust based on the number of unique classes
    'eval_metric': 'mlogloss',      # Multi-class log loss
    'max_depth': 6,                 # Limits tree depth to prevent overfitting
    'learning_rate': 0.05,          # Step size shrinkage
    'subsample': 0.8,               # Prevents overfitting by using 80% of data per tree
    'colsample_bytree': 0.8,        # Uses 80% of features per tree
    'random_state': 42
}

In [ ]:
evals = [(dtrain, 'train'), (dval, 'validation')]  # Evaluation datasets

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=200,  # Number of boosting rounds (trees)
    evals=evals,
    early_stopping_rounds=10,  # Stops if validation loss doesn't improve for 10 rounds
    verbose_eval=True  # Shows training progress
)

In [ ]:
y_pred = model.predict(dval)  # Predictions on validation set


In [ ]:
y_test_pred = model.predict(dtest)  # Predictions on test set


In [ ]:
y_test_pred

In [ ]:
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')


In [ ]:

xgb.plot_importance(model)
plt.show()